In [5]:
import torch
import soundfile as sf
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2ProcessorWithLM
import torchvision
import torchaudio
import subprocess
import sox

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
# wav2vec2_model_name = "facebook/wav2vec2-base-960h" # 360MB
wav2vec2_model_name = "Harveenchadha/vakyansh-wav2vec2-hindi-him-4200"
# wav2vec2_model_name = "facebook/wav2vec2-large-960h-lv60-self" # pretrained 1.26GB
# wav2vec2_model_name = "jonatasgrosman/wav2vec2-large-xlsr-53-english" # English-only, 1.26GB
# wav2vec2_model_name = "jonatasgrosman/wav2vec2-large-xlsr-53-arabic" # Arabic-only, 1.26GB
# wav2vec2_model_name = "jonatasgrosman/wav2vec2-large-xlsr-53-spanish" # Spanish-only, 1.26GB

wav2vec2_processor = Wav2Vec2Processor.from_pretrained(wav2vec2_model_name)
wav2vec2_model = Wav2Vec2ForCTC.from_pretrained(wav2vec2_model_name).to(device)
wave2vec_processor_with_LM = Wav2Vec2ProcessorWithLM.from_pretrained(wav2vec2_model_name)

c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at Harveenchadha/vakyansh-wav2vec2-hindi-him-4200 were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to 

In [17]:
wav2vec2_model.device

AttributeError: 'Wav2Vec2ProcessorWithLM' object has no attribute 'device'

In [12]:
sp,sr = sf.read("t1.mp3")

In [14]:
sp.shape

(221440,)

### This part is only for wav2vec LM and hindi model

In [26]:
def read_file_and_process(wav_file, processor):
    filename = wav_file.split('.')[0]
    filename_16k = filename + "16k.wav"
    resampler(wav_file, filename_16k)
    speech, _ = sf.read(wav_file)
    inputs = processor(speech, sampling_rate=16_000, return_tensors="pt", padding=True).to(device)
    
    return inputs


def resampler(input_file_path, output_file_path):
    command = (
        f"ffmpeg -hide_banner -loglevel panic -i {input_file_path} -ar 16000 -ac 1 -bits_per_raw_sample 16 -vn "
        f"{output_file_path}"
    )
    subprocess.call(command, shell=True)


def parse_transcription_with_lm(logits, processor_with_LM):
    result = processor_with_LM.batch_decode(logits.cpu().numpy())
    text = result.text
    transcription = text[0].replace('<s>','')
    return transcription

def parse_transcription(logits, processor):
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0], skip_special_tokens=True)
    return transcription

def speech_to_text_parse(wav_file, applyLM, read_file_and_process, model, processor_with_LM, processor):
    input_values = read_file_and_process(wav_file, processor)
    with torch.no_grad():
        logits = model(**input_values).logits
   
    if applyLM:
        return parse_transcription_with_lm(logits, processor_with_LM)
    else:
        return parse_transcription(logits, processor)

In [27]:
speech_to_text_parse(wav_file="t1.mp3", applyLM=True, read_file_and_process=read_file_and_process, model=wav2vec2_model,
                     processor_with_LM=wave2vec_processor_with_LM, processor=wav2vec2_processor) # type: ignore

Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.


'      '

In [48]:
# Second Try
import subprocess
import torch
import soundfile as sf
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, Wav2Vec2ProcessorWithLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def read_file_and_process(mp3_file, processor):
    # Load the MP3 file
    speech, sample_rate = torchaudio.load(mp3_file)
    
    # Resample if necessary
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        speech = resampler(speech)
    
    # Process the audio
    inputs = processor(speech.squeeze().numpy(), sampling_rate=16000, return_tensors="pt", padding=True).to(device)
    return inputs

def parse_transcription_with_lm(logits, processor_with_LM):
    try:
        result = processor_with_LM.batch_decode(logits.cpu().numpy())
        text = result[0][0]
        transcription = text.replace('<s>', '')
    except NotImplementedError:
        print("Parallel batch decoding is not supported. Falling back to sequential decoding.")
        transcription = parse_transcription(logits, processor_with_LM)
    return transcription

def parse_transcription(logits, processor):
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]
    transcription = transcription.replace('<s>', '')
    return transcription

def speech_to_text_parse(mp3_file, applyLM, model, processor_with_LM, processor):
    input_values = read_file_and_process(mp3_file, processor)
    with torch.no_grad():
        logits = model(**input_values).logits
   
    if applyLM:
        return parse_transcription_with_lm(logits, processor_with_LM)
    else:
        return parse_transcription(logits, processor)

In [49]:
# Example usage
wav2vec2_model_name = "Harveenchadha/vakyansh-wav2vec2-hindi-him-4200"
processor = Wav2Vec2Processor.from_pretrained(wav2vec2_model_name)
processor_with_LM = Wav2Vec2ProcessorWithLM.from_pretrained(wav2vec2_model_name)
model = Wav2Vec2ForCTC.from_pretrained(wav2vec2_model_name).to(device)

c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Fetching 4 files: 100%|██████████| 4/4 [00:00<?, ?it/s]
Only 0.0% of unigrams in vocabulary found in kenlm model-- this might mean that your vocabulary and language model are incompatible. Is this intentional?
Some weights of the model checkpoint at Harveenchadha/vakyansh-wav2vec2-hindi-him-4200 were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another archit

In [55]:
mp3_file = "t1.mp3"
applyLM = True  # Set to False if you don't want to use the language model
transcription = speech_to_text_parse(mp3_file, applyLM, model, processor_with_LM, processor)
print("Transcription:", transcription)

Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.


Transcription:        


In [ ]:
# def parse_transcription_with_lm(logits):
#     result = processor_with_LM.batch_decode(logits.cpu().numpy())
#     text = result.text
#     transcription = text[0].replace('<s>','')
#     return transcription

# def parse_transcription(logits):
#     predicted_ids = torch.argmax(logits, dim=-1)
#     transcription = processor.decode(predicted_ids[0], skip_special_tokens=True)
#     return transcription

# def parse(wav_file, applyLM):
#     input_values = read_file_and_process(wav_file)
#     with torch.no_grad():
#         logits = model(**input_values).logits
   
#     if applyLM:
#         return parse_transcription_with_lm(logits)
#     else:
#         return parse_transcription(logits)

    
model_id = "Harveenchadha/vakyansh-wav2vec2-hindi-him-4200"
processor = Wav2Vec2Processor.from_pretrained(model_id)
processor_with_LM = Wav2Vec2ProcessorWithLM.from_pretrained(model_id)
model = Wav2Vec2ForCTC.from_pretrained(model_id)

In [18]:
from IPython.display import Audio
Audio("sample-0.mp3")

In [5]:

from IPython.display import Audio
Audio("marathi.mp3")

In [9]:
audio_url = "https://github.com/x4nth055/pythoncode-tutorials/raw/master/machine-learning/speech-recognition/30-4447-0004.wav"

In [6]:

def load_audio(audio_path):
  """Load the audio file & convert to 16,000 sampling rate"""
  # load our wav file
  speech, sr = torchaudio.load(audio_path)
  resampler = torchaudio.transforms.Resample(sr, 16000)
  speech = resampler(speech)
  return speech.squeeze()

In [8]:
# load audio and pad/trim it to fit 30 seconds
#audio = whisper.load_audio("t1.mp3")
audio = load_audio("marathi.mp3")
# tokenize our wav
input_values = wav2vec2_processor(audio, return_tensors="pt", sampling_rate=16000)["input_values"].to(device)
input_values.shape

# Performing the inference of data
# Lets pass the vector into our model and retrive the logits
# perform inference
logits = wav2vec2_model(input_values)["logits"]
logits.shape

# Get the predictions from the logits
# Passing the logits to torch.argmax() to get the likely prediction:
# use argmax to get the predicted IDs
predicted_ids = torch.argmax(logits, dim=-1)
predicted_ids.shape

# decode the IDs to text

transcription = wav2vec2_processor.decode(predicted_ids[0])
#transcription.lower()

# print the recognized text
print(transcription)

APLAMUCH GUD TYP KARAKI MA PEST KARAJA MULETO ATSHIPEC SHAJAST ABAT BHASHA ANI U CHARAN MADHESARVAT SAN VADAT MAK MACH GUD TEBHASHANAT RU PANTRITUI


In [41]:
import gradio as gr 
import time
import torchaudio

c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [54]:
def transcribe(audio, state=""):
    
    def load_audio(audio_path):
        """Load the audio file & convert to 16,000 sampling rate"""
        # load our wav file
        speech, sr = torchaudio.load(audio_path)
        resampler = torchaudio.transforms.Resample(sr, 16000)
        speech = resampler(speech)
        return speech.squeeze()
    time.sleep(2)
    # load audio and pad/trim it to fit 30 seconds
    audio = load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")

    # decode the audio
    #time.sleep(2)
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    state += result.text + " "
    return state

In [55]:
gr.Interface(
    title = 'OpenAI Whisper ASR Gradio Web UI', 
    fn=transcribe, 
    inputs=[
        gr.Audio(sources=["microphone"], type="filepath")
    ],
    outputs=[
        "textbox"
    ],
    live=True).launch()

Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


Detected language: en


Traceback (most recent call last):
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\gradio\queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\gradio\route_utils.py", line 288, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\gradio\blocks.py", line 1931, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\gradio\blocks.py", line 1516, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_

Detected language: en


Traceback (most recent call last):
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\gradio\queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\gradio\route_utils.py", line 288, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\gradio\blocks.py", line 1931, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\gradio\blocks.py", line 1516, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_

...

Traceback (most recent call last):
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\gradio\queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\gradio\route_utils.py", line 288, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\gradio\blocks.py", line 1931, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\gradio\blocks.py", line 1516, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_

Detected language: en


Traceback (most recent call last):
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\gradio\queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\gradio\route_utils.py", line 288, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\gradio\blocks.py", line 1931, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\gradio\blocks.py", line 1516, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_

Detected language: en


Traceback (most recent call last):
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\gradio\queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\gradio\route_utils.py", line 288, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\gradio\blocks.py", line 1931, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_Projects\myenv\Lib\site-packages\gradio\blocks.py", line 1516, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anupa\OneDrive\Desktop\Eneru_